# Легенда

Мы работаем дата-сайентистом в компании Booking. Одна из проблем компании — это нечестные отели, которые накручивают себе рейтинг. Одним из способов обнаружения таких отелей является построение модели, которая предсказывает рейтинг отеля. Если предсказания модели сильно отличаются от фактического результата, то, возможно, отель ведёт себя нечестно, и его стоит проверить.

Нам поставлена задача создать такую модель.

# Загрузка библиотек и данных

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# импортируем библиотеки для визуализации
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# для нормализации, стандартизации
from sklearn import preprocessing

# импортируем библиотеку для работы с кодировщиками
import category_encoders as ce

# импортируем объект Counter из модуля collections
from collections import Counter

# импортируем класс SentimentIntensityAnalyzer из библиотеки nltk
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.downloader.download('vader_lexicon')

# загружаем специальный удобный инструмент для разделения датасета
from sklearn.model_selection import train_test_split

# библиотеки для оценки значимости
from sklearn.feature_selection import chi2 # хи-квадрат
from sklearn.feature_selection import f_classif # anova

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [ ]:
# Подгрузим наши данные из соревнования

DATA_DIR = '/kaggle/input/sf-booking/'
df_train = pd.read_csv(DATA_DIR+'/hotels_train.csv') # датасет для обучения
df_test = pd.read_csv(DATA_DIR+'hotels_test.csv') # датасет для предсказания
sample_submission = pd.read_csv(DATA_DIR+'/submission.csv') # самбмишн

In [ ]:
df_train.info()

In [ ]:
df_train.head(2)

In [ ]:
df_test.info()

In [ ]:
df_test.head(2)

In [ ]:
sample_submission.info()

In [ ]:
sample_submission.head(2)

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['reviewer_score'] = 0 # в тесте у нас нет значения reviewer_score, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
data.info()

# Создание новых признаков

#### 1. Работа с колонкой 'hotel_address'

In [ ]:
# посмотрим, какую информацию содержит в себе колонка 'hotel_address'
list(data['hotel_address'].unique())[:10]

In [ ]:
# создаём функцию для извлечения города из адреса
def get_city(address):
    address_list = address.split(' ')
    if address_list[-2] == 'United' and address_list[-1] == 'Kingdom':
        return address_list[-5]
    else:
        return address_list[-2]

# создаём функцию для извлечения страны из адреса
def get_country(address):
    address_list = address.split(' ')
    if address_list[-2] == 'United' and address_list[-1] == 'Kingdom':
        return address_list[-2] + ' ' + address_list[-1]
    else:
        return address_list[-1]

# создаём колонки с названиями городов и стран
data['city'] = data['hotel_address'].apply(get_city)
data['country'] = data['hotel_address'].apply(get_country)

# проверяем полученные данные
print('Города:', data['city'].unique())
print('Страны:', data['country'].unique())

In [ ]:
# извлекаем из Википедии информацию о населении и площади имеющихся у нас городов и стран
city_data = pd.DataFrame({
    'city': ['Milan', 'Amsterdam', 'Barcelona', 'London', 'Paris', 'Vienna'],
    'city_population': [1378689, 872757, 1636732, 8961989, 2148327, 1897491],
    'city_area': [181.67, 219.4, 101.3, 1602, 105.4, 414.75]
})
country_data = pd.DataFrame({
    'country': ['Italy', 'Netherlands', 'Spain', 'United Kingdom', 'France', 'Austria'],
    'country_population': [59236213, 17665300, 46552504, 67081000, 68084217, 8923507],
    'country_area': [302073, 41543, 505990, 242495, 643801, 83879]
})

# добавим колонки с плотностью населения
city_data['city_density'] = (city_data['city_population'] / city_data['city_area']).astype('int')
country_data['country_density'] = (country_data['country_population'] / country_data['country_area']).astype('int')

# добавляем полученные данные к датасету с отелями
data = data.join(city_data.set_index('city'), on = 'city')
data = data.join(country_data.set_index('country'), on = 'country')

In [ ]:
# создадим дополнительную колонку-индикатор, находится ли отель в столице
capitals = ['Amsterdam', 'London', 'Paris', 'Vienna']
data['is_capital'] = data['city'].apply(lambda x: 1 if x in capitals else 0)

In [ ]:
# визуализируем распределение данных по городам
city_data = data['city'].value_counts()
fig = plt.figure(figsize=(5, 5))
axes = fig.add_axes([0, 0, 1, 1])
axes.pie(
    city_data,
    labels = city_data.index,
    autopct = '%.1f%%',
    explode = [0.1, 0, 0, 0, 0, 0]
);

#### 2. Работа с колонкой 'review_date'

In [ ]:
# преобразовываем колонку даты в формат datetime
data['review_date'] = pd.to_datetime(data['review_date'])

# извлекаем день недели из даты
data['day_of_week'] = data['review_date'].dt.dayofweek

# извлекаем месяц из даты
data['month'] = data['review_date'].dt.month

In [ ]:
# создаём и используем функцию для извлечения сезона
def get_season(date):
    if date.month in [1, 2, 12]:
        return 'winter'
    if date.month in [3, 4, 5]:
        return 'spring'
    if date.month in [6, 7, 8]:
        return 'summer'
    else:
        return 'autumn'
data['season'] = data['review_date'].apply(get_season)

In [ ]:
# визуализируем распределение данных по сезонам
season_data = data['season'].value_counts()
fig = plt.figure(figsize=(5, 5))
axes = fig.add_axes([0, 0, 1, 1])
axes.pie(
    season_data,
    labels = season_data.index,
    autopct = '%.1f%%',
    explode = [0.1, 0, 0, 0]
);

#### 3. Работа с колонкой 'hotel_name'

In [ ]:
# поищем найболее полезные слова в названиях отелей
words = []
for hotel in data['hotel_name'].values:
    words.extend(hotel.split(' '))

# создаём объект Counter и передаём в него полученные слова для подсчёта
word_counts = Counter(words)

# с помощью метода most_common() наблюдаю найболее популярные слова
word_counts.most_common()[:10]

In [ ]:
# выберем 15 слов, которые на мой взгляд могут на что-то влиять
words_to_apply = [
    'park', 'hilton', 'plaza', 'grand', 'inn', 'city', 'holiday', 'western',
    'palace', 'radisson', 'blu', 'mercure', 'tower', 'paddington', 'novotel'
]

# перебираем выбранные слова и создаём колонки-индикаторы, указывающие, содеражат ли отели эти слова в названии
for word in words_to_apply:
    data[word + '_hotel'] = data['hotel_name'].apply(
        lambda x: 1 if word in x.lower() else 0
    )

#### 4. Работа с колонкой 'reviewer_nationality'

In [ ]:
# для начала очистим значения от лишних пробелов
data['reviewer_nationality'] = data['reviewer_nationality'].apply(lambda x: x.strip())

# отмечаем является ли рецензент гражданином страны, в которой находится отель
data['is_citizen'] = data['reviewer_nationality'] == data['country']
data['is_citizen'] = data['is_citizen'].astype('int')

# оставим в списке только 15 самых популярных национальностей, остальные заполним значением 'other'
top_nations = list(data['reviewer_nationality'].value_counts()[:15].index)
data['reviewer_nationality'] = data['reviewer_nationality'].apply(lambda x: x if x in top_nations else 'other')

In [ ]:
# визуализируем распределение данных по гражданству
reviewer_data = data['reviewer_nationality'].value_counts()
fig = plt.figure(figsize=(5, 5))
axes = fig.add_axes([0, 0, 1, 1])
axes.pie(
    reviewer_data,
    labels = reviewer_data.index,
    autopct = '%.1f%%'
);

#### 5. Работа с колонками 'negative_review' и 'positive_review'

In [ ]:
# посмотрим на самые популярные отзывы
display(data['negative_review'].value_counts()[:10])
print() # отступ
display(data['positive_review'].value_counts()[:10])

Для выражения негативных и позитивных отзывов в числовом формате будем использовать класс SentimentIntensityAnalyzer из библиотеки nltk. Для его коректной работы необходимы некоторые преобразования в значениях нашего датасета.

In [ ]:
# уберём лишние пробелы и приведём все символы к нижнему регистру
data['negative_review'] = data['negative_review'].apply(
    lambda x: x.strip().lower()
)
data['positive_review'] = data['positive_review'].apply(
    lambda x: x.strip().lower()
)

# отсутствие отзывов дающее эмоциональную окраску заполняем пустыми строками
not_negative = [
    'no negative', 'no complaints', 'nothing all good', 'having to leave',
    'nothing it was perfect', 'nothing everything was perfect',
    'there was nothing i didn t like', 'no'
]
not_positive = ['no positive']

data['negative_review'] = data['negative_review'].apply(
    lambda x: '' if x in not_negative else x
)
data['positive_review'] = data['positive_review'].apply(
    lambda x: '' if x in not_positive else x
)

# популярные значения, имеющие эмоциональную окраску в зависимости от того, в какой
# колонке они находятся, дополним суффиксом в соответствии с названием колонки
real = [
    'location', 'the location', 'everything', 'small room', 'breakfast',
    'price', 'small rooms', 'staff', 'location and staff', 'location staff',
    'location location location', 'comfy bed', 'expensive', 'room size',
    'expensive breakfast', 'the price', 'the staff', 'all', 'every thing',
    'cleanliness', 'location breakfast', 'location only', 'wifi', 'pillows',
    'parking', 'size of room', 'the breakfast', 'breakfast too expensive',
    'central location', 'staff and location', 'the bed', 'convenient location',
    'position', 'location and breakfast'
]

data['positive_review'] = data['positive_review'].apply(
    lambda x: x + ' positive' if x in real else x
)
data['negative_review'] = data['negative_review'].apply(
    lambda x: x + ' negative' if x in real else x
)

In [ ]:
# создаём класс SentimentIntensityAnalyzer
sent_analyzer = SentimentIntensityAnalyzer()

# создаём колонки с числовым выражением отзывов
data['negative_neg'] = data['negative_review'].apply(
    lambda x: sent_analyzer.polarity_scores(x)['neg']
)
data['negative_neu'] = data['negative_review'].apply(
    lambda x: sent_analyzer.polarity_scores(x)['neu']
)
data['negative_pos'] = data['negative_review'].apply(
    lambda x: sent_analyzer.polarity_scores(x)['pos']
)
data['negative_compound'] = data['negative_review'].apply(
    lambda x: sent_analyzer.polarity_scores(x)['compound']
)
data['positive_neg'] = data['positive_review'].apply(
    lambda x: sent_analyzer.polarity_scores(x)['neg']
)
data['positive_neu'] = data['positive_review'].apply(
    lambda x: sent_analyzer.polarity_scores(x)['neu']
)
data['positive_pos'] = data['positive_review'].apply(
    lambda x: sent_analyzer.polarity_scores(x)['pos']
)
data['positive_compound'] = data['positive_review'].apply(
    lambda x: sent_analyzer.polarity_scores(x)['compound']
)

#### 6. Работа с колонкой 'tags'

In [ ]:
# создаём полный список тэгов
tags = []
for tag_list in data['tags'].values:
    tag_list = tag_list.replace("[' ", "")
    tag_list = tag_list.replace(" ']", "")
    tag_list = tag_list.split(" ', ' ")
    tags.extend(tag_list)

# подсчитываем количество тэгов с помощью класса Counter
tag_counter = Counter(tags)
sorted_tags = list(tag_counter.most_common())
sorted_tags[:10]

In [ ]:
# создаём функцию и извлекаем количество ночей, проведённых в отеле
def get_stayed_nights(tags):
    tags_list = tags.split(' ')
    if 'Stayed' in tags_list:
        night_index = tags_list.index('Stayed') + 1
        return int(tags_list[night_index])
    else:
        return np.NaN

data['stayed_nights'] = data['tags'].apply(get_stayed_nights)


# дополним датасет ещё 15 колонками-индикаторами самых популярных тэгов
new_columns = []

for tag, count in sorted_tags:
    if 'Stayed' not in tag:
        new_columns.append(tag)    
        
for column in new_columns[:15]:
    data[column] = data['tags'].apply(lambda x: 1 if column in x else 0)

#### 7. Работа с колонкой 'days_since_review'

In [ ]:
# разбиваем колонку по пробелу и оставляем только числовую часть данных
data['days_since_review'] = data['days_since_review'].apply(
    lambda x: int(x.split(' ')[0])
)

#### 8. Кодирование признаков

In [ ]:
# Номинальные признаки содержащие не более 16 уникальных значений, кодируем используя класс OneHotEncoding.
encoder = ce.OneHotEncoder(
    cols = ['city', 'day_of_week', 'month', 'season', 'reviewer_nationality']
)
data_bin = encoder.fit_transform(
    data[['city', 'day_of_week', 'month', 'season', 'reviewer_nationality']]
)
data = pd.concat([data, data_bin], axis = 1)

# Очистка от пропущенных значений

In [ ]:
# заполним пропуски в географических координатах модальным значением в зависимости от города
data['lat'] = data['lat'].fillna(
    data.groupby(['city'])['lat'].transform(lambda x: x.mode()[0])
)
data['lng'] = data['lng'].fillna(
    data.groupby(['city'])['lng'].transform(lambda x: x.mode()[0])
)

# пропущенные значения в колонке 'stayed_nights' заполняем модальным значением
data['stayed_nights'].fillna(data['stayed_nights'].mode()[0], inplace=True)
# и заменяем тип данных на 'int'
data['stayed_nights'] = data['stayed_nights'].astype('int')

# Удаление строковых значений

In [ ]:
# убираем признаки которые еще не успели обработать, 
# модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
object_columns = [s for s in data.columns if data[s].dtypes == 'object']
data.drop(object_columns, axis = 1, inplace = True)

# удаляем колонку с типом datetime
data.drop('review_date', axis = 1, inplace = True)

# удаляем колонки 'day_of_week' и 'month', которые уже закодированы как категориальные
data = data.drop(['day_of_week', 'month'], axis = 1)

# Преобразование признаков

Так как у нас получилось очень много бинарных признаков (со значением 0 или 1), то и остальные признаки будет логичным привести к этому диапозону.

In [ ]:
# сохраняем колонки которые будем преобразовывать
col_names = []
for col in data.columns:
    if (data[col].min() < 0 or data[col].max() > 1) and col != 'reviewer_score':
        col_names.append(col)

# выводим полученный результат
print('Колонки к преобразованию:')
for col in col_names:
    print(col)

In [ ]:
# инициализируем нормализатор MinMaxScaler
mm_scaler = preprocessing.MinMaxScaler()

# создаём нормализованные колонки
data_mm = mm_scaler.fit_transform(data[col_names])

# преобразуем промежуточный датасет в полноценный датафрейм
data_mm = pd.DataFrame(data_mm, columns = col_names)

# удаляем преобразованные колонки из исходного датасета
data = data.drop(col_names, axis = 1)

# объеденяем данные
data = pd.concat([data_mm, data], axis = 1)

# Отбор признаков

#### 1. Анализ мультиколлинеарности

In [ ]:
# для корреляции с категориальными признаками будем использовать метод Спирмена
corr = data.drop(['sample'], axis=1).corr(method = 'spearman')

# выведем только данные с высоким уровнем корреляции
mask_1 = np.abs(corr) >= 0.70
mask_2 = np.abs(corr) <= -0.70
corr_data = corr[mask_1 | mask_2]

fig = plt.figure(figsize=(20, 20))
axes = fig.add_axes([0, 0, 1, 1])
sns.heatmap(corr_data, annot=True, ax=axes, linewidth=0.3, linecolor='black');

In [ ]:
# чтобы понять какие признаки удалить, посмотрим как они коррелируют с целевым признаком 'reviewer_score'
corr = data[[
    'additional_number_of_scoring', 'total_number_of_reviews', 'lat', 'lng',
    'city_population', 'city_area', 'city_density', 'country_area',
    'country_density', 'is_capital', 'inn_hotel', 'holiday_hotel',
    'radisson_hotel', 'blu_hotel', 'is_citizen', 'Leisure trip',
    'Business trip', 'city_3', 'city_4', 'reviewer_nationality_1',
    'reviewer_score'
]].corr(method = 'spearman')

fig = plt.figure(figsize=(15, 10))
axes = fig.add_axes([0, 0, 1, 1])
sns.heatmap(corr, annot=True, ax=axes, linewidth=0.3, linecolor='black');

In [ ]:
# удаляем те признаки, которые меньше коррелируют с целевым признаком
data = data.drop([
    'radisson_hotel', 'city_density', 'country_area', 'is_citizen',
    'is_capital', 'lat', 'inn_hotel', 'city_population', 'city_4',
    'additional_number_of_scoring', 'Business trip'
], axis = 1)

#### 2. Оценка значимости признаков

In [ ]:
# разбиваем датафрейм на части, необходимые для оценки значимости
# X - данные с информацией об отелях, y - целевая переменная (рейтинги отелей)
X = data.drop(['reviewer_score'], axis = 1)
y = data['reviewer_score'].astype('int')


# разделяем признаки на непрерывные и категориальные
num_cols = [
    'average_score', 'review_total_negative_word_counts',
    'total_number_of_reviews', 'review_total_positive_word_counts',
    'total_number_of_reviews_reviewer_has_given', 'days_since_review',
    'city_area', 'country_population', 'country_density', 'negative_compound',
    'positive_compound', 'stayed_nights', 'negative_neg', 'negative_neu',
    'negative_pos', 'positive_neg', 'positive_neu', 'positive_pos'
]
cat_cols = [
    'lng', 'park_hotel', 'hilton_hotel', 'plaza_hotel',
    'grand_hotel', 'city_hotel', 'holiday_hotel', 'western_hotel',
    'palace_hotel', 'blu_hotel', 'mercure_hotel',
    'tower_hotel', 'paddington_hotel', 'novotel_hotel',
    'Leisure trip', 'Submitted from a mobile device', 'Couple',
    'Solo traveler', 'Group', 'Family with young children',
    'Double Room', 'Standard Double Room', 'Superior Double Room',
    'Family with older children', 'Deluxe Double Room', 'Double or Twin Room',
    'Standard Double or Twin Room', 'Classic Double Room', 'city_1', 'city_2',
    'city_3', 'city_5', 'city_6', 'day_of_week_1', 'day_of_week_2',
    'day_of_week_3', 'day_of_week_4', 'day_of_week_5', 'day_of_week_6',
    'day_of_week_7', 'month_1', 'month_2', 'month_3', 'month_4', 'month_5',
    'month_6', 'month_7', 'month_8', 'month_9', 'month_10', 'month_11',
    'month_12', 'season_1', 'season_2', 'season_3', 'season_4',
    'reviewer_nationality_1', 'reviewer_nationality_2',
    'reviewer_nationality_3', 'reviewer_nationality_4',
    'reviewer_nationality_5', 'reviewer_nationality_6',
    'reviewer_nationality_7', 'reviewer_nationality_8',
    'reviewer_nationality_9', 'reviewer_nationality_10',
    'reviewer_nationality_11', 'reviewer_nationality_12',
    'reviewer_nationality_13', 'reviewer_nationality_14',
    'reviewer_nationality_15', 'reviewer_nationality_16'
]

In [ ]:
# для оценки значимости категориальных переменных будем использовать непараметрический тест хи-квадрат
imp_cat = pd.Series(chi2(X[cat_cols], y)[0], index = cat_cols)
imp_cat.sort_values(inplace = True)

# визуализируем полученные результаты
fig = plt.figure(figsize=(10, 10))
axes = fig.add_axes([0, 0, 1, 1])
axes.barh(imp_cat.index, imp_cat)
axes.set_title('Значимость категориальных переменных')
axes.set_xlabel('Значение хи-квадрат');

In [ ]:
# оставим только найболее значимую половину категориальных признаков
cols_to_drop = imp_cat[imp_cat.values < imp_cat.median()]
data = data.drop(cols_to_drop.index, axis = 1)

In [ ]:
# для оценки значимости непрерывных переменных будем использовать функцию f_classif
imp_num = pd.Series(f_classif(X[num_cols], y)[0], index = num_cols)
imp_num.sort_values(inplace = True)

# визуализируем полученные результаты
fig = plt.figure()
axes = fig.add_axes([0, 0, 1, 1])
axes.barh(imp_num.index, imp_num)
axes.set_title('Значимость непрерывных переменных')
axes.set_xlabel('Значения f-cтатистики');

In [ ]:
# оставим только найболее значимую половину числовых признаков
cols_to_drop = imp_num[imp_num.values < imp_num.median()]
data = data.drop(cols_to_drop.index, axis = 1)

In [ ]:
data.info()

# Этап машинного обучения

In [ ]:
# Теперь выделим тестовую часть
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample'], axis=1)

y = train_data.reviewer_score.values            # наш таргет
X = train_data.drop(['reviewer_score'], axis=1)

In [ ]:
# Воспользуемся специальной функцией train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh');

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['reviewer_score'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission

In [ ]:
list(sample_submission)

In [ ]:
sample_submission['reviewer_score'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)